In [1]:
# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from imutils import build_montages
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [2]:
from transformers import AutoFeatureExtractor,TFViTMAEModel

features_model = AutoFeatureExtractor.from_pretrained("facebook/vit-mae-base")

In [3]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images("comofod"))
data = []
labels = []
for imagePath in tqdm(imagePaths):
    if imagePath.split('\\')[1] == 'morphed':
        label = imagePath.split(os.path.sep)[-2]
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (224, 224))
        input_ = features_model(image,return_tensors="tf")['pixel_values'].numpy()[0]
        nimage = cv2.resize(input_,(224,224))
        nimage = input_.T
        if len(data) <= 2500:
            data.append(nimage)
            labels.append(label)
    else:
        label = imagePath.split(os.path.sep)[-2]
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (224, 224))
        input_ = features_model(image,return_tensors="tf")['pixel_values'].numpy()[0]
        nimage = cv2.resize(input_,(224,224))
        nimage = input_.T
        if len(data) <= 5000:
            data.append(nimage)
            labels.append(label)

[INFO] loading images...


100%|████████████████████████████████████████████████████████████████████████████| 10400/10400 [07:50<00:00, 22.11it/s]


In [4]:
data = np.array(data)#, dtype="float") / 255.0
data = data.reshape((data.shape[0], data.shape[1], data.shape[2], 3))
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 2)

In [5]:
print(data.shape, labels.shape)

(5001, 224, 224, 3) (5001, 2)


In [6]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.45, stratify=labels, random_state=42)

In [7]:
opt = Adam(lr=1e-4, decay=1e-4 / 5)

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [9]:
from tensorflow.keras.applications import ResNet50

model = ResNet50(weights = 'imagenet', include_top=True, input_shape=(224, 224, 3))
output = Dense(2, activation='softmax')(model.layers[-2].output)
model = Model(model.input, output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [10]:
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

In [11]:
from tensorflow.keras.callbacks import TensorBoard

tensorboard = TensorBoard(log_dir='./logs',write_graph=True, write_images=False)

In [12]:
H = model.fit(x=trainX, y=trainY, validation_data=(testX, testY),batch_size=4, callbacks=[tensorboard], epochs=10, verbose=1)

Epoch 1/10
688/688 [==============================] - 56s 65ms/step - loss: 0.6944 - accuracy: 0.6578 - val_loss: 0.4845 - val_accuracy: 0.7339
Epoch 2/10


ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv5_block1_0_conv/Conv2D' defined at (most recent call last):
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\ProgramData\Anaconda3\lib\asyncio\base_events.py", line 596, in run_forever
      self._r

In [ ]:
model.save("comfod_forge.h5")

In [ ]:
y_pred = model.predict(testX)

In [ ]:
def plot_confusion_matrix(test_y, predict_y):
    C = confusion_matrix(test_y, predict_y)   
    A =(((C.T)/(C.sum(axis=1))).T)    
    B =(C/C.sum(axis=0))
    plt.figure(figsize=(20,4))    
    labels = [0,1]
    cmap=sns.light_palette("blue")    
    plt.subplot(1, 3, 2)
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Precision matrix")    
    plt.subplot(1, 3, 3)
    # representing B in heatmap format
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Recall matrix")    
    plt.show()

In [ ]:
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(testY, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

plot_confusion_matrix(y_test, y_pred)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model

#model.summary()
IMAGE_PATH = 'test/casia2_morphed.jpg'
LAYER_NAME = 'conv5_block3_out'

img =cv2.imread(IMAGE_PATH)
image = cv2.resize(img, (224, 224))
input_ = features_model(image,return_tensors="tf")['pixel_values'].numpy()[0]
nimage = cv2.resize(input_,(224,224))
nimage = input_.T
grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(LAYER_NAME).output, model.output])

with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(np.array([nimage]))
    loss = predictions[:,np.argmax(predictions)]

output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]  * 1e+05
gate_f = tf.cast(output > 0, 'float32')
gate_r = tf.cast(grads > 0, 'float32')
guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

weights = tf.reduce_mean(guided_grads, axis=(0, 1))

cam = np.ones(output.shape[0: 2], dtype = np.float32)

for i, w in enumerate(weights):
    cam += w * output[:, :, i]

cam = cv2.resize(cam.numpy(), (img.shape[1], img.shape[0]))
denom = (cam.max() - cam.min()) #+ 1e-16
cam = np.maximum(cam, 0)
heatmap = (cam - cam.min()) / denom
cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
print(img.shape, cam.shape)
output_image = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, cam, 1, 0)

cv2.imwrite('test.png', output_image)

In [ ]:
from IPython.display import Image 
pil_img = Image(filename='test.png')
display(pil_img)